In [1]:
from zipfile import ZipFile
import os

base_folder = "./data"
file_name = os.path.join(base_folder, 'brains.zip')
data_folder = os.path.join(base_folder, 'brains')

with ZipFile(file_name, 'r') as zip:
    zip.extractall(data_folder)


In [4]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow as tf
import os

folder_path = os.path.join(data_folder, 'Brain Tumor Data Set/Brain Tumor Data Set')

train_ds = image_dataset_from_directory(
    folder_path,
  validation_split=0.2,
  seed=123,
    subset='training',
  image_size=(512, 512),
  batch_size=30)

val_ds = image_dataset_from_directory(
    folder_path,
  validation_split=0.2,
  seed=123,
  image_size=(512, 512),
    subset='validation',
  batch_size=30)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(1)
val_ds = val_ds.prefetch(1)

Found 4514 files belonging to 2 classes.
Using 3612 files for training.
Found 4514 files belonging to 2 classes.
Using 902 files for validation.


In [9]:
from keras import models
from keras import layers

model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-6),
    loss='binary_crossentropy',
    metrics=['acc']
)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 510, 510, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 255, 255, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 253, 253, 64)      18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 126, 126, 64)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 124, 124, 128)     73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 62, 62, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 60, 60, 128)      

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

filename = 'Brain.h5'

checkpoint = ModelCheckpoint(filename,             
                             monitor='val_loss',   
                             verbose=1,            
                             save_best_only=True,  
                             mode='auto'           
                            )

earlystopping = EarlyStopping(monitor='val_loss',  
                              patience=10,         
                             )


In [11]:
history =  model.fit(
    train_ds,
    epochs=60,
    validation_data=val_ds,
    callbacks=[checkpoint, earlystopping],
)

Epoch 1/60
121/121 [==============================] - 24s 192ms/step - loss: 0.9883 - acc: 0.6938 - val_loss: 0.5543 - val_acc: 0.7738

Epoch 00001: val_loss improved from inf to 0.55428, saving model to Brain.h5
Epoch 2/60
121/121 [==============================] - 24s 195ms/step - loss: 0.4245 - acc: 0.8206 - val_loss: 0.4180 - val_acc: 0.8359

Epoch 00002: val_loss improved from 0.55428 to 0.41802, saving model to Brain.h5
Epoch 3/60
121/121 [==============================] - 24s 194ms/step - loss: 0.2740 - acc: 0.8840 - val_loss: 0.2934 - val_acc: 0.8869

Epoch 00003: val_loss improved from 0.41802 to 0.29340, saving model to Brain.h5
Epoch 4/60
121/121 [==============================] - 23s 192ms/step - loss: 0.1855 - acc: 0.9341 - val_loss: 0.2967 - val_acc: 0.8847

Epoch 00004: val_loss did not improve from 0.29340
Epoch 5/60
121/121 [==============================] - 23s 189ms/step - loss: 0.1423 - acc: 0.9543 - val_loss: 0.2016 - val_acc: 0.9290

Epoch 00005: val_loss improved

Epoch 40/60
121/121 [==============================] - 23s 189ms/step - loss: 0.0014 - acc: 1.0000 - val_loss: 0.0633 - val_acc: 0.9789

Epoch 00040: val_loss did not improve from 0.06266
Epoch 41/60
121/121 [==============================] - 23s 190ms/step - loss: 0.0014 - acc: 1.0000 - val_loss: 0.0631 - val_acc: 0.9789

Epoch 00041: val_loss did not improve from 0.06266
Epoch 42/60
121/121 [==============================] - 23s 191ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.0648 - val_acc: 0.9789

Epoch 00042: val_loss did not improve from 0.06266
Epoch 43/60
121/121 [==============================] - 23s 189ms/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0617 - val_acc: 0.9789

Epoch 00043: val_loss improved from 0.06266 to 0.06166, saving model to Brain.h5
Epoch 44/60
121/121 [==============================] - 23s 189ms/step - loss: 0.0010 - acc: 1.0000 - val_loss: 0.0626 - val_acc: 0.9789

Epoch 00044: val_loss did not improve from 0.06166
Epoch 45/60
121/121 [=========